<a href="https://colab.research.google.com/github/fasterinnerlooper/Notebooks/blob/main/training/train-codeberta-on-lcc_csharp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
%pip install torch
import torch
if not torch.cuda.is_available():
  %pip install tensorflow==2.14 intel-extension-for-tensorflow[xpu]==2.14 intel-optimization-for-horovod==0.28.1.1 torch==2.0.1 torchvision==0.15.2 intel_extension_for_pytorch==2.0.100 oneccl-bind-pt==2.0.0 --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/ -q


In [108]:
%pip install wandb numpy datasets transformers[torch] accelerate -U -q

In [11]:
use_wandb = True

if use_wandb:
    import wandb

    wandb.init(project="codeBERTa-on-csharp")

wandb: Currently logged in as: shafiq-jetha. Use `wandb login --relogin` to force relogin


In [109]:
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
)


model_name = "huggingface/CodeBERTa-small-v1"

# download prepare the data
# dataset = load_dataset("code_search_net","javascript")
dataset = load_dataset("fasterinnerlooper/LCC_csharp")
tokenizer = AutoTokenizer.from_pretrained(model_name, return_special_tokens_mask=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

In [110]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True
)

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [131]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(30000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(10000))

In [103]:
small_train_dataset.features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [135]:
import os

model = AutoModelForMaskedLM.from_pretrained(model_name)
model.to("cuda")

if wandb:
    os.environ["WANDB_PROJECT"] = "codeBERTa-on-csharp"
    os.environ["WANDB_LOG_MODEL"] = "true"
    os.environ["WANDB_WATCH"] = "false"

# pass "wandb" to the 'report_to' parameter to turn on wandb logging

training_args = TrainingArguments(
    output_dir="models",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=500,
    # push_to_hub=True,
)

if wandb:
    training_args.report_to = ["wandb"]

# define the trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [136]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.990400,2.116087
2,1.735800,1.929185


Epoch,Training Loss,Validation Loss
1,1.990400,2.116087
2,1.735800,1.929185
3,1.650300,1.854758


TrainOutput(global_step=11250, training_loss=1.940063197157118, metrics={'train_runtime': 2935.7293, 'train_samples_per_second': 30.657, 'train_steps_per_second': 3.832, 'total_flos': 2984110571520000.0, 'train_loss': 1.940063197157118, 'epoch': 3.0})

In [ ]:
model.push_to_hub()

In [134]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

if wandb:
    wandb.finish()

RuntimeError: ignored

In [130]:
from transformers import pipeline

text = """
//
// Author:
//   Jb Evain (jbevain@gmail.com)
//
// Copyright (c) 2008 - 2015 Jb Evain
// Copyright (c) 2008 - 2011 Novell, Inc.
//
// Licensed under the MIT/X11 license.
//

namespace Mono.Cecil.Cil {

	public sealed class VariableDefinition : VariableReference {

		public bool IsPinned {
			get { return variable_type.IsPinned; }
		}

		public VariableDefinition (TypeReference variableType)
			: base (variableType)
		{
		}

		public override VariableDefinition Resolve ()
		{
			<mask> <mask>;
		}
	}
}"""

# should produce 'return this'
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device="cuda")
res = pipe(text)
for r in res[0]:
	if(r['score'] > .50):
		for x in (pipe(r['sequence'])):
			print(str(x['score']) + ": " + str(x['token_str']))
			print(r['token_str'] + x['token_str'])

0.2009483426809311:  override
 return override
0.18098247051239014:  false
 return false
0.11529174447059631:  true
 return true
0.06685683131217957:  undefined
 return undefined
0.055736079812049866:  resolve
 return resolve
